In [1]:
from vllm import LLM, SamplingParams
from dotenv import load_dotenv

load_dotenv()

# llama-3-70 quantized
llm = LLM("meta-llama/Llama-3.2-3B-Instruct")
#llm = LLM('meta-llama/Llama-3.2-1B-Instruct', gpu_memory_utilization=0.9, tensor_parallel_size=8, enforce_eager=False, dtype="half")



/home/rog0d/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-12 21:00:58,848	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 11-12 21:01:03 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-12 21:01:03 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-12 21:01:03 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.50it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  2.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.97it/s]


INFO 11-12 21:03:07 model_runner.py:1067] Loading model weights took 6.0160 GB


INFO 11-12 21:03:07 gpu_executor.py:122] # GPU blocks: 37315, # CPU blocks: 2340
INFO 11-12 21:03:07 gpu_executor.py:126] Maximum concurrency for 131072 tokens per request: 4.56x
INFO 11-12 21:03:10 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-12 21:03:10 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-12 21:03:22 model_runner.py:1523] Graph capturing finished in 12 secs.


In [21]:
import random
random_seed = random.randint(0, 10000)
random.seed(random_seed)

print(f"seed: {random_seed}")

uvl_grammar_no_indent = r"""
?start: _NL* featuremodel

featuremodel: "features" _NL [feature+]
feature: NAME _NL (group+)?

group: "or" groupspec          -> or_group
    | "alternative" groupspec -> alternative_group
    | "optional" groupspec    -> optional_group
    | "mandatory" groupspec   -> mandatory_group
    | cardinality groupspec    -> cardinality_group

groupspec: _NL feature+

cardinality: "[" INT (".." (INT | "*"))? "]"

%import common.INT
%import common.CNAME -> NAME
%import common.WS_INLINE
%ignore WS_INLINE

_NL: /\r?\n[ \t]*/
"""

uvl_grammar = r"""
?start: _NL* featuremodel

featuremodel: "features" _NL ["->" feature+ "<-"]
feature: NAME _NL ("->" group+ "<-")?

group: "or" groupspec          -> or_group
    | "alternative" groupspec -> alternative_group
    | "optional" groupspec    -> optional_group
    | "mandatory" groupspec   -> mandatory_group
    | cardinality groupspec    -> cardinality_group

groupspec: _NL "->" feature+ "<-"

cardinality: "[" INT (".." (INT | "*"))? "]"

%import common.INT
%import common.CNAME -> NAME
%import common.WS_INLINE
%ignore WS_INLINE

_NL: /\r?\n[ \t]*/
"""

uvl_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for creating sintactically correct expressions similar to these examples:

features
    Sandwich
        mandatory
            Bread
        optional
            Sauce
                alternative
                    Ketchup
                    Mustard
            Cheese
constraints
    Ketchup => Cheese
,
features
	SmartWatch
		mandatory
			Functionalities 
				mandatory
					FitnessMonitor
					SleepTracker
					VibrateAlert
						mandatory
							Call
							Notification
			Sensors
				mandatory
					Pedometer
					Accelerometer
				optional
					HeartRateSensor
			Connectivity
				mandatory
					BT40


<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Write new, valid expressions for a computer: 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

arithmetic_grammar = """
?start: comparison

?comparison: expression ("=" expression)?

?expression: term (("+" | "-") term)*

?term: factor (("*" | "/") factor)*

?factor: NUMBER
       | "-" factor
       | "(" comparison ")"

%import common.NUMBER
%ignore " "  // Ignore spaces

"""
arithmetic_prompt="Rewrite 5*5 as another expression"

arithmetic_prompt_fewshots="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant for creating gramatically, equivalent and correct arithmetical expression<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Rewrite the following expressions into equivalent ones as show in this example(5*5)=(5+5+5+5+5)=(25*1)=(5*3)+(5*2).\n(3*3)=\n(3*4*5)=\n(7*3)=
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

sql_grammar="""
start: select_statement
select_statement: "SELECT" column "from" table "where" condition
column: "col_1" | "col_2"
table: "table_1" | "table_2"
condition: column "=" number
number: "1" | "2"
"""
sql_prompt="Generate a sql state that select col_1 from table_1 where it is equals to 1"

import time
start_time = time.perf_counter()
grammar = uvl_grammar_no_indent
prompt = uvl_prompt
# llama-3 8B
#MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
#llm = LLM(MODEL, gpu_memory_utilization=1, tensor_parallel_size=8, enforce_eager=False, dtype="half") 


sampling_params = SamplingParams(
        max_tokens=500,
        temperature=1,
        top_p=0.95,
        min_tokens=200,
    )


outputs = llm.generate(
    prompts=prompt,
    sampling_params=sampling_params,
    guided_options_request=dict(guided_grammar=grammar))

elapsed_time = time.perf_counter() - start_time
print(f'Elapsed time for inference: {elapsed_time} seconds')

from lark import Lark, exceptions
from lark.indenter import Indenter


def test(generation: str, parser):
    print(parser.parse(generation).pretty())

"""
class TreeIndenter(Indenter):
    NL_type = '_NL'
    OPEN_PAREN_types = []
    CLOSE_PAREN_types = []
    INDENT_type = '_INDENT'
    DEDENT_type = '_DEDENT'
    tab_len = 8
"""

#parser = Lark(grammar, parser='lalr', postlex=TreeIndenter())
parser = Lark(grammar, parser='lalr')

print("Grammar is well-written.")

for output in outputs:
    prompt = output.prompt

    generated_text = output.outputs[0].text
    print(f"Generated text without parser: {generated_text!r}")

    try:
        # Parse a generation
        test(generation=generated_text, parser=parser)

    except:
        print("Generation not grammatically valid")

seed: 1725


Processed prompts: 100%|██████████| 1/1 [01:43<00:00, 103.94s/it, est. speed input: 1.67 toks/s, output: 4.81 toks/s]

Elapsed time for inference: 104.51963531202637 seconds
Grammar is well-written.
Generated text without parser: 'features\n    CPU\n       mandatory\n           ClockSpeed\n       optional\n           HyperThreading\n               alternative\n                   TurboBoost\n                   IntegratedGraphics\n       Memory\n           mandatory\n               RAM\n       optional\n                   Storage\n                       alternative\n                           SSDHDD\n                           NVMeSSD\n   PowerSupply\n       mandatory\n           Voltage\n       optional\n           PowerSharing\n               alternative\n                   Overclocking\n   Display\n       mandatory\n           Resolution\n       optional\n                   Brightness\n                       alternative\n                           Touchscreen\n                           HDRDisplay\n   OperatingSystem\n       mandatory\n           Stability\n       optional\n                   Security

In [22]:
print(f"nº of generated tokens: {str(len(outputs[0].outputs[0].token_ids))}")
print(f"generated tokens id: {outputs[0].outputs[0].token_ids}")


nº of generated tokens: 500
generated tokens id: (20922, 198, 262, 220, 32715, 198, 286, 81216, 198, 310, 27196, 11303, 198, 286, 13099, 198, 310, 75046, 1016, 6285, 198, 394, 78095, 198, 504, 38062, 754, 53463, 198, 504, 83537, 21309, 198, 286, 10869, 198, 310, 81216, 198, 394, 35005, 198, 286, 13099, 198, 504, 5913, 198, 667, 78095, 198, 1014, 1242, 52426, 4195, 198, 1014, 37426, 7979, 1242, 35, 198, 262, 15335, 52396, 198, 286, 81216, 198, 310, 95443, 198, 286, 13099, 198, 310, 15335, 84856, 198, 394, 78095, 198, 504, 1959, 21321, 287, 198, 262, 7165, 198, 286, 81216, 198, 310, 39206, 198, 286, 13099, 198, 504, 94347, 198, 667, 78095, 198, 1014, 11561, 8337, 198, 1014, 71388, 7165, 198, 262, 59247, 2374, 198, 286, 81216, 198, 310, 626, 2968, 198, 286, 13099, 198, 504, 15712, 198, 667, 78095, 198, 1014, 10480, 2065, 3883, 198, 1014, 37196, 24264, 19855, 198, 262, 79107, 198, 286, 81216, 198, 310, 14953, 1968, 198, 286, 13099, 198, 504, 38945, 1752, 198, 667, 78095, 198, 1014, 47182, 

In [23]:
gen = outputs[0].outputs[0].text
print(f"{str(gen)}")

print(parser.parse(gen).pretty())



features
    CPU
       mandatory
           ClockSpeed
       optional
           HyperThreading
               alternative
                   TurboBoost
                   IntegratedGraphics
       Memory
           mandatory
               RAM
       optional
                   Storage
                       alternative
                           SSDHDD
                           NVMeSSD
   PowerSupply
       mandatory
           Voltage
       optional
           PowerSharing
               alternative
                   Overclocking
   Display
       mandatory
           Resolution
       optional
                   Brightness
                       alternative
                           Touchscreen
                           HDRDisplay
   OperatingSystem
       mandatory
           Stability
       optional
                   Security
                       alternative
                           CustomizationOptions
                           BiometricAuthentication
   Networking

UnexpectedToken: Unexpected token Token('$END', '') at line 142, column 24.
Expected one of: 
	* _NL
